In [1]:
import pandas as pd
import numpy as np
#import h2o
#from h2o.estimators.gbm import H2OGradientBoostingEstimator
#from h2o.estimators.random_forest import H2ORandomForestEstimator

In [2]:
train = pd.read_pickle('../data/preproc/interactions_train_preproc_rare.pickle')
#test = pd.read_pickle('../data/preproc/test_interaction_test_preproc.pkl')

In [3]:
for item in train.select_dtypes(include=['category']).columns:
    train[item] = train[item].cat.codes
#    test[item] = test[item].cat.codes

In [4]:
y = train['target']
train = train.drop('target', axis=1)

In [5]:
from sklearn.model_selection import train_test_split

In [ ]:
import lightgbm

In [6]:
#categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categoricals = train.select_dtypes(include=['int8', 'int16', 'int32']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(train, y, random_state=101)
del train, y

### lightGBM

In [11]:
model_gbm = lightgbm.LGBMClassifier(
    seed=np.random.randint(10**10),
    n_estimators=200,
    max_depth=10,
    num_leaves=200,
    min_data_in_leaf=100,
    subsample=0.8,
    colsample_bytree=0.8,
    #is_unbalance=True,
    objective='binary'
)

model_gbm.fit(X_train_train, y_train_train, 
              eval_set=[(X_train_val, y_train_val)], 
              eval_metric='auc', 
              categorical_feature=categorical_features_indices)

/home/julien/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1042: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [34, 35]
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


MemoryError: 

### SGDclassifier

In [7]:
from sklearn.preprocessing import OneHotEncoder
#from scipy.sparse import csr_matrix,csc_matrix
from sklearn.cross_validation import StratifiedKFold
#from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

/home/julien/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#kfolder = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=1)

#grand_auc = 0

#X = np.array(train)

model = SGDClassifier(loss='log', penalty='l1', alpha=0.5, max_iter=50, random_state=1)

one=OneHotEncoder(handle_unknown='ignore', categorical_features=categorical_features_indices)
one.fit(X_train_train)
X_train_train=one.transform(X_train_train)
X_train_cv=one.transform(X_train_val)
model.fit(X_train_train,y_train_train)
preds=model.predict_proba(X_train_val)[:,1]
auc=roc_auc_score(y_train_val,preds)

In [16]:
h2o.init(nthreads=2, max_mem_size = "16G")             #specify max number of bytes. uses all cores by default.
h2o.remove_all() #clean slate, in case cluster was already running

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 mins 27 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.5
H2O cluster version age:,29 days
H2O cluster name:,H2O_from_python_julien_33ct88
H2O cluster total nodes:,1
H2O cluster free memory:,7.101 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [17]:
train_H2O = h2o.H2OFrame(train)
#test_h2O = h2o.H2OFrame(test)

/home/julien/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


MemoryError: 

In [ ]:
#X_train, X_valid, X_test = train_H2O.split_frame([0.60, 0.20], seed=1234)
X_train, X_valid = train_H2O.split_frame([0.75, seed=1234)

In [ ]:
X = train_H2O.col_names     #last column is Cover_Type, our desired response variable 
X.remove('target')
y = 'target'

In [ ]:
params = {}

rf = H2ORandomForestEstimator(
    model_id="rf_v1",
    ntrees=100,
    stopping_rounds=2,
    score_each_iteration=False,
    
    seed=1000000)

In [ ]:
rf.train(X, y, training_frame=X_train, validation_frame=X_valid)